In [1]:
from bs4 import BeautifulSoup
import os
import json

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
save_dir = r'/home/alex/data/wir/'
source_dir = r'raffineurs'
save_dir = os.path.join(save_dir, source_dir)
base_url= 'https://www.lesraffineurs.com/'

In [6]:
from utils import get_p_html, save_img

# DL images

In [232]:
with open('prods_w_images_temp.json', 'w') as fp:
    json.dump(prods, fp)

In [231]:
%%time
for prod in prods[:3]:
    print(prod['name'])
    for img_url in prod['images']:
        url = img_url
        img_name = '-'.join([prod['id']]+url.split('/')[-2:])
        print('\t'+img_name)
        image_local_path = os.path.join(
            save_dir
            ,img_name)
        save_img(url, image_local_path)

T-shirt Hot Robot
	1591-13041-large_default-t-shirt-hot-robot.jpg
	1591-13042-large_default-t-shirt-hot-robot.jpg
Horloge des marées
	552-9740-large_default-horloge-des-marees.jpg
	552-9739-large_default-horloge-des-marees.jpg
	552-13551-large_default-horloge-des-marees.jpg
	552-13553-large_default-horloge-des-marees.jpg
	552-13552-large_default-horloge-des-marees.jpg
	552-9737-large_default-horloge-des-marees.jpg
	552-9738-large_default-horloge-des-marees.jpg
Baromètre à cristaux
	542-4526-large_default-barometre-a-cristaux.jpg
	542-4525-large_default-barometre-a-cristaux.jpg
	542-4527-large_default-barometre-a-cristaux.jpg


# product page scraping

In [215]:
url = 'du-temps-libre/552-horloge-des-marees.html'
url = os.path.join(base_url, url)
htmlBytes, _ = get_p_html(url)
soup = BeautifulSoup(htmlBytes, 'html.parser')

get image URLs

In [216]:
#images = image_soup[0].text.split('\n')[1:-1]

In [217]:
from scrap_prod_image_links import get_images_from_soup

In [218]:
images = get_images_from_soup(soup)

In [219]:
images

['https://lesraffineurs.b-cdn.net/9740-large_default/horloge-des-marees.jpg',
 'https://lesraffineurs.b-cdn.net/9739-large_default/horloge-des-marees.jpg',
 'https://lesraffineurs.b-cdn.net/13551-large_default/horloge-des-marees.jpg',
 'https://lesraffineurs.b-cdn.net/13553-large_default/horloge-des-marees.jpg',
 'https://lesraffineurs.b-cdn.net/13552-large_default/horloge-des-marees.jpg',
 'https://lesraffineurs.b-cdn.net/9737-large_default/horloge-des-marees.jpg',
 'https://lesraffineurs.b-cdn.net/9738-large_default/horloge-des-marees.jpg']

get all text

In [109]:
def get_cleaned_text(soup):
    # source: https://stackoverflow.com/questions/1936466/beautifulsoup-grab-visible-webpage-text
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text.encode('utf-8')

In [110]:
text = get_cleaned_text(soup)

build script

In [207]:
with open('prods.json', 'r') as fp:
    prods = json.load(fp)

max_prods = 3
count_prods = 0
for prod in prods:
    print(prod['name'])
    count_prods +=1

    # fetch HTML code
    url = prod['href']
    htmlBytes, _ = get_p_html(url)
    soup = BeautifulSoup(htmlBytes, 'html.parser')
    
    # extract image links
    prod['images'] = get_images_from_soup(soup)
    
    # a bit of cleaning
    prod['thumb_img'] = prod.pop('image')
    prod['category'] = url.split('/')[-2]

    if count_prods >= max_prods:
        break
        
#with open('prods_w_images.json', 'w') as fp:
#    json.dump(prods, fp)

T-shirt Hot Robot
Horloge des marées
Baromètre à cristaux


In [208]:
prods[2]

{'href': 'https://www.lesraffineurs.com/du-temps-libre/542-barometre-a-cristaux.html',
 'name': 'Baromètre à cristaux',
 'id': '542',
 'images': ['https://lesraffineurs.b-cdn.net/4526-large_default/barometre-a-cristaux.jpg',
  'https://lesraffineurs.b-cdn.net/4525-large_default/barometre-a-cristaux.jpg',
  'https://lesraffineurs.b-cdn.net/4527-large_default/barometre-a-cristaux.jpg'],
 'thumb_img': 'https://lesraffineurs.b-cdn.net/4526-productlist/barometre-a-cristaux.jpg',
 'category': 'du-temps-libre'}

# product list scraping

In [71]:
prods = []
pages = range(1,9)

max_prods = 10e9
prod_count = 0

for page in pages:
    
    print('page: {}'.format(page))
    url = '22-tous-les-produits?page={}'.format(page)
    url = os.path.join(base_url, url)
    htmlBytes, _ = get_p_html(url)
    soup = BeautifulSoup(htmlBytes, 'html.parser')
    soup_products = soup.findAll(attrs={'class': 'thumbnail-container'})
    
    for prod in soup_products:
        
        info = prod.findAll('a')
        
        prod = {}
        prod['href'], prod['name'] = info[1].get("href"), info[1].text
        prod['image'] = info[0].findAll('source')[0].get('data-srcset')
        prod['id'] = prod['href'].split('/')[-1].split('-')[0]
        prods.append(prod)
        print('\t{}'.format(prod['name']))
        
        prod_count += 1
        
        if prod_count >  max_prods:
            break
            
    if prod_count >  max_prods:
            break
        
with open('prods.json', 'w') as fp:
    json.dump(prods, fp)

page: 1
	T-shirt Hot Robot
	Horloge des marées
	Baromètre à cristaux
